# COGS 118A - Final Project

### Prediction of Chess Game Outcomes Based on Player Features

# Names

Hopefully your team is at least this good.

- Omri Habot
- Keate Ehrenburg
- Jacob Lamadrid
- Alex Bumbalov

# Abstract

The goal of this project is to examine the effectiveness of different models in predicting the outcome of a chess match. The data used in this project will contain features describing players’ technical skill and experience including the Elo rating of each player, the number of games they have played, the number of wins/losses/draws they have achieved, their age, and the length of the game. These are measured by observing the outcomes of every game in a player’s career. We will use this data to perform feature selection and single out the most relevant features, and then we will train the models and compare their predictive accuracies. Accuracy will be measured by classification error metrics including precision, recall, f1-score, and possibly others, which we will compare across models to observe their relative efficacies.

# Background

Within the world of chess competitions and general play, machine learning and deep learning have been famously applied in such algorithms as Deep Blue, Stockfish, etc. These algorithms historically have aimed to predict the next best move to be executed as well as the win probability at any given position state [1](https://arxiv.org/pdf/2109.11602.pdf). This win probability is what we aim to place our project emphasis upon as in similar works in varying competitive settings, most relevant of which is found in esports win/loss classification based upon player/team rankings and typical movesets among many other features [2](https://arxiv.org/pdf/2108.02799.pdf). The application of these algorithms in returning win probability and outcome prediction has large impacts in the way one chooses to learn chess or the way in which a machine is taught how to play chess. This may also have implications for the way in which new strategies or entire play styles are formed. These implications have already manifested themselves in the competitive playspace as many players look towards Stockfish evaluations for input on their play or for other engines in which competition may aid in their training as “with the help of chess engines, Grandmasters are now able to plan prepare for their games in extreme depth, sometimes memorizing up to 15-20 moves of their openings” [3](https://fluency.mcsaatchi.com/2022/09/01/the-evolution-of-chess-ai/#:~:text=Artificial%20intelligence%20has%20completely%20changed,20%20moves%20of%20their%20openings.). In order to approach the problem of finding an appropriate machine learning prediction method , we are going to need a large amount of chess match data to work with. We have chosen to use the Free Internet Chess Server(FICS) database. The database has been collecting all rated and unrated games played on FICS since October of 2008. With over 200 million games stored, this will provide us with plenty of data to address our goal of outcome classification.

Our models will train similarly to that of current chess engines, utilizing previous games played in order to gain improve decision making and outcome prediction, as in the models created by the chess.com website, with machine learning algorithms such as Luk.AI with improves its elo with every game played.

# Problem Statement

The problem we will be aiming to address is determining which machine learning method best predicts the winner of a chess game depending on variables like participating players, player statistics and scores such as their Elo’s, the opening, and psychological factors of the players, among others. We plan to explore this problem by trying three different machine learning algorithms: logistic regression, random forest, and deep neural network. Logistic regression is a solid classification method. While we fear that it may be too simple for this task, we believe it is a simple baseline to compare with other, more complex models. We plan to try random forests because the complex decision boundaries may be sufficient for consistently accurate and generalizable predictions. Finally, we plan on using a deep neural network because it seems to be the baseline method used for very similar problems, and given our large dataset, we hope it will pick up on important nuances that are neglected by logistic regression and random forest. We will use labeled chess game data that denotes the winner of each game as well as the aforementioned variables that describe the game. Each model will be trained using the same (or similar) dataset and evaluated with the same set of classification error metrics such that we can compare their efficacies. These metrics include precision, recall, f1-score, and others.

# Data
# __NEEDS WORK__ - Dataset is small, a larger one is suggested. Is label encoding or one hot encoding on the chess game type appropriate for our solution? If so, can we redesign our dataframe in this way? If not, why?
We will be using the [Free Internet Chess Server (FICS)](https://www.ficsgames.org/dl/ficsgamesdb_202301_blitz2000_movetimes_278527.pgn.bz2) as our database for analysis. The dataset comes in the Portable Game Notation (PGN) file format, which is a common data format for representing chess games. We will need to convert this into a pandas dataframe for our analysis, so we will write a function that parses through the PGN file as text and builds a dataframe from it. Once the data is parsed, we will simply need to convert each column into the appropriate datatype, so numbers would be int32 or float64, words will be strings, binary features will be boolean, etc.

There are many different types of chess games from which we can choose that would cause nuances in our analysis and results. We selected blitz games because their rapid nature produces more variability in game moves, and therefore outcomes. We hope that this variability gives us more data to work with when predicting outcomes. Our dataset has 2461 observations (each representing a chess game) as well as 20 features describing each game. Each observation consists of the moves made throughout the game, the date and time played, the players and their colors, player Elo, match result, and game variant and time control information for each match. One of the most critical features is `playerElo`, a score representing the relative skill levels of players, since this is likely one of the biggest predictors of the outcome. This is represented by an arbitrary number that can indicate a player's relative skill level when compared to other players. Another crucial feature is `moves`, which outlines all the moves made in the game. We can use this sequential data to build more complex models and make more nuanced predictions.

### Data Importing/Cleaning

In [ ]:
import pandas as pd
import re

def parse_game(game):
    sections = re.split('\n\s*\n', game.strip())
    game_dict = dict([line.replace("]", '').replace("[", '').split(' ', 1) for line in sections[0].split('\n')])
    game_dict['Moves'] = sections[1]
    return game_dict

with open('ficsgamesdb_202301_blitz2000_movetimes_278527.pgn') as f:
    #next(f)
    games = (parse_game(game) for game in f.read().split('\n\n['))

In [ ]:
df = pd.DataFrame(games)
df.head()

,Event,Site,FICSGamesDBGameNo,White,Black,WhiteElo,BlackElo,WhiteRD,BlackRD,WhiteIsComp,TimeControl,Date,Time,WhiteClock,BlackClock,ECO,PlyCount,Result,Moves,BlackIsComp
0,"""FICS rated blitz game""","""FICS freechess.org""","""530203389""","""konozrout""","""Geforce""","""2060""","""1957""","""27.4""","""15.9""","""Yes""","""300+0""","""2023.01.31""","""23:27:00""","""0:05:00.000""","""0:05:00.000""","""B00""","""55""","""1/2-1/2""",1. e4 {[%emt 0.0]} e5 {[%emt 0.0]} 2. d4 {[%em...,NaN
1,"""FICS rated blitz game""","""FICS freechess.org""","""530203352""","""Geforce""","""pikozrout""","""1951""","""2137""","""15.9""","""39.5""",NaN,"""300+0""","""2023.01.31""","""23:17:00""","""0:05:00.000""","""0:05:00.000""","""B40""","""69""","""1/2-1/2""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,"""Yes"""
2,"""FICS rated blitz game""","""FICS freechess.org""","""530203324""","""Geforce""","""konozrout""","""1957""","""2054""","""15.9""","""27.5""",NaN,"""300+0""","""2023.01.31""","""23:03:00""","""0:05:00.000""","""0:05:00.000""","""B52""","""134""","""0-1""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,"""Yes"""
3,"""FICS rated blitz game""","""FICS freechess.org""","""530203266""","""GimmeDatKing""","""Frubes""","""2292""","""1796""","""74.1""","""49.4""",NaN,"""300+0""","""2023.01.31""","""22:43:00""","""0:05:00.000""","""0:05:00.000""","""B90""","""55""","""1-0""",1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,NaN
4,"""FICS rated blitz game""","""FICS freechess.org""","""530203175""","""playoften""","""GimmeDatKing""","""1826""","""2290""","""24.8""","""74.5""",NaN,"""300+0""","""2023.01.31""","""22:22:00""","""0:05:00.000""","""0:05:00.000""","""A04""","""42""","""0-1""",1. Nf3 {[%emt 0.0]} f5 {[%emt 0.0]} 2. d4 {[%e...,NaN


In [ ]:
# Cleaning

def clean_df(df): # Depricated
    datetime_columns = ['Date', 'Time', 'WhiteClock', 'BlackClock']
    float_columns = ['WhiteRD', 'BlackRD']
    int_columns = ['FICSGamesDBGameNo', 'WhiteElo', 'BlackElo', 'PlyCount']
    
    for column in df.columns:
        
        if df[column].dtype != object or not isinstance(df.iloc[0][column], str): # Keeps from cleaning the data twice
            continue
        if column in float_columns:
            df[column] = df[column].apply(lambda f: float(f.replace('"', '')))
        elif column in int_columns:
            df[column] = df[column].apply(lambda n: int(n.replace('"', '')))
        elif column in datetime_columns:
            df[column] = df[column].apply(lambda d: pd.to_datetime(d.replace('"', '')))
        else:
            df[column] = df[column].apply(lambda s: str(s).replace('"', '')).astype('category')
    
    df['WhiteIsComp'] = df['WhiteIsComp'] == 'Yes'
    df['BlackIsComp'] = df['BlackIsComp'] == 'Yes'
    
    return df

In [ ]:
cleaned = clean_df(df)
print(cleaned.shape)
print(cleaned.dtypes)
cleaned.head()

(2461, 20)
Event                      category
Site                       category
FICSGamesDBGameNo             int64
White                      category
Black                      category
WhiteElo                      int64
BlackElo                      int64
WhiteRD                     float64
BlackRD                     float64
WhiteIsComp                    bool
TimeControl                category
Date                 datetime64[ns]
Time                 datetime64[ns]
WhiteClock           datetime64[ns]
BlackClock           datetime64[ns]
ECO                        category
PlyCount                      int64
Result                     category
Moves                      category
BlackIsComp                    bool
dtype: object


,Event,Site,FICSGamesDBGameNo,White,Black,WhiteElo,BlackElo,WhiteRD,BlackRD,WhiteIsComp,TimeControl,Date,Time,WhiteClock,BlackClock,ECO,PlyCount,Result,Moves,BlackIsComp
0,FICS rated blitz game,FICS freechess.org,530203389,konozrout,Geforce,2060,1957,27.4,15.9,True,300+0,2023-01-31,2023-03-09 23:27:00,2023-03-09 00:05:00,2023-03-09 00:05:00,B00,55,1/2-1/2,1. e4 {[%emt 0.0]} e5 {[%emt 0.0]} 2. d4 {[%em...,False
1,FICS rated blitz game,FICS freechess.org,530203352,Geforce,pikozrout,1951,2137,15.9,39.5,False,300+0,2023-01-31,2023-03-09 23:17:00,2023-03-09 00:05:00,2023-03-09 00:05:00,B40,69,1/2-1/2,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
2,FICS rated blitz game,FICS freechess.org,530203324,Geforce,konozrout,1957,2054,15.9,27.5,False,300+0,2023-01-31,2023-03-09 23:03:00,2023-03-09 00:05:00,2023-03-09 00:05:00,B52,134,0-1,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
3,FICS rated blitz game,FICS freechess.org,530203266,GimmeDatKing,Frubes,2292,1796,74.1,49.4,False,300+0,2023-01-31,2023-03-09 22:43:00,2023-03-09 00:05:00,2023-03-09 00:05:00,B90,55,1-0,1. e4 {[%emt 0.0]} c5 {[%emt 0.0]} 2. Nf3 {[%e...,False
4,FICS rated blitz game,FICS freechess.org,530203175,playoften,GimmeDatKing,1826,2290,24.8,74.5,False,300+0,2023-01-31,2023-03-09 22:22:00,2023-03-09 00:05:00,2023-03-09 00:05:00,A04,42,0-1,1. Nf3 {[%emt 0.0]} f5 {[%emt 0.0]} 2. d4 {[%e...,False


# Proposed Solution
# __NEEDS WORK__ - Could we provide an explanation on why these models are suitable solutions for our problem?
In this project we aim to evaluate the effectiveness of three different machine learning implementations at classifying the outcomes of FIDE titled chess games from unlabeled match data. Each model (Logisitc Regression, Random Forests, and the Deep NN) will be trained on a curated sample database consisting of several derived features representing characteristics of the games pertinent to the games’ outcomes. For this we will need to use a variety of tools for accessing and manipulating the data to generate these special features. Pandas and numpy will be instrumental in this preprocessing stage. Further imports will be required for sklearn and matplot lib for training/validation and analysis, as well as an additional library called pychess will be necessary for processing chess engine ratings and to help format game data for preprocessing. We will be using built in methods for training and validating data provided to us by sklearn. To measure the effectiveness of our model we will take Sayon Bhattacharjee’s LSTM solution published in the _Towards Data Science_ article found [here](https://towardsdatascience.com/predicting-professional-players-chess-moves-with-deep-learning-9de6e305109e). Sayon's LTSM solution uses two convolutional neural networks; one "predicts" where a particular piece moved from, and the other predicts where a piece will move. These networks are combined as a time series to model a classification problem with three possible output categories corresponding with the outcome: black wins, white wins, or there is a draw. The effectiveness of this model can simply be measured by classification accuracy using metrics discussed in the following section.

# Evaluation Metrics

In the evaluation of binary chess outcome classification, simply implementing a confusion matrix of our algorithm’s results is most appropriate for visualizing the performance. In this situation, there is equal cost associated with both false positive rates and false negative rates, and there is equal cost associated with true positives and true negative rates. Consequently, we care about both the recall and precision of our models equally. This suggests that an f1-score is one primary metric we should use, since it is equally sensitive to both recall and precision. For example, predicting that Player 1 wins when they actually win is equally as important as being ‘accurate’ when predicting that Player 1 will win; the cost of a false positive is no different than the cost of a false negative. We may also explore some other classification error metrics such as likelihood ratios.

$$precision = \frac{TP}{TP + FP}$$
$$recall = \frac{TP}{TP + FN}$$
$$F1-Score = \frac{2*precision*recall}{precision + recall}$$

# Results
# __NEEDS WORK__
You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result
# __NEEDS WORK__
OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations
# __NEEDS WORK__
Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

In the case of data privacy, we see no major concerns with our data as it is publicly available FIDE match information that is made public anytime a chess player agrees to a FIDE match. The biggest ethical concern that we could potentially see with the production of any well-functioning model for predicting FIDE titled chess matches would be the use of the model for gambling purposes. Although we don't foresee the prediction accuracy of our models being able to give a definitive answer to who will win a match, a good model could lead to an increase in chess match gambling which comes with its own host of harmful consequences. Additionally, we can highlight the concern that a prediction model could affect the way that players go about their matches if they know the predicted outcome of a game given specific features. This could be seen as an unfair advantage. The model, if then re-trained with new data, could also perform much more poorly as outcomes would now be affected by prior knowledge of the potential outcome of a game which would complicate further predictions.

### Conclusion
# __NEEDS WORK__
Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.